# Setup Network Streaming for CVBS Capture

This notebook configures and starts the Red Pitaya streaming server for continuous CVBS capture over the network.

## Advantages over DMA Capture
- **Unlimited duration** - stream for hours (limited by disk space on receiving PC)
- **Real-time** - data streams to PC as captured
- **No reboot required** - works with default DMA memory allocation

## Data Rate Requirements
- Sample rate: 15.625 MS/s (decimation 8)
- Data rate: 31.25 MB/s (16-bit samples)
- Network capacity: 62.5 MB/s max (well within limits)

## 1. Check OS Version

Network streaming at full rate requires OS 2.07-43 or newer.

In [ ]:
import subprocess
import os

# Check Red Pitaya OS version
try:
    with open('/opt/redpitaya/version.txt', 'r') as f:
        version = f.read().strip()
    print(f"Red Pitaya OS Version: {version}")
    
    # Parse version to check if streaming is supported at full rate
    parts = version.split('-')
    if len(parts) >= 2:
        major_minor = parts[0]
        if major_minor >= '2.07':
            print("✓ OS version supports 62.5 MB/s network streaming")
        elif major_minor >= '1.04':
            print("⚠ OS version supports up to 20 MB/s network streaming")
            print("  Consider upgrading to OS 2.07-43+ for full rate")
        else:
            print("✗ OS version may not support streaming")
except FileNotFoundError:
    print("Could not determine OS version")

## 2. Check Streaming Server Availability

In [ ]:
# Check if streaming server binary exists
streaming_paths = [
    '/opt/redpitaya/bin/streaming-server',
    '/opt/redpitaya/bin/rpsa_server',
]

server_path = None
for path in streaming_paths:
    if os.path.exists(path):
        server_path = path
        print(f"✓ Found streaming server: {path}")
        break

if not server_path:
    print("✗ Streaming server not found")
    print("  You may need to use the web interface Streaming application instead")

## 3. Configure Streaming Parameters

In [ ]:
# Streaming configuration for CVBS capture
config = {
    'decimation': 8,           # 125/8 = 15.625 MS/s
    'channels': 1,             # Single channel (IN1)
    'resolution': 16,          # 16-bit (14-bit ADC stored in 16-bit)
    'protocol': 'TCP',         # Reliable transport
    'port': 18900,             # Default ADC streaming port
}

sample_rate = 125e6 / config['decimation']
data_rate = sample_rate * (config['resolution'] / 8) * config['channels']

print("Streaming Configuration:")
print(f"  Decimation: {config['decimation']}")
print(f"  Sample rate: {sample_rate/1e6:.3f} MS/s")
print(f"  Channels: {config['channels']}")
print(f"  Resolution: {config['resolution']}-bit")
print(f"  Data rate: {data_rate/1e6:.2f} MB/s")
print(f"  Port: {config['port']}")

## 4. Create Configuration File

In [ ]:
# Create streaming configuration file
config_content = f"""# Red Pitaya Streaming Server Configuration
# For CVBS capture at 15.625 MS/s

[streaming]
decimation = {config['decimation']}
channels = CH1
resolution = 16
protocol = TCP
port = {config['port']}
format = RAW
"""

config_path = '/tmp/cvbs_stream_config'
with open(config_path, 'w') as f:
    f.write(config_content)

print(f"Configuration written to {config_path}")
print()
print(config_content)

## 5. Start Streaming Server

**IMPORTANT:** Run the receiver script on your PC FIRST, then run this cell to start streaming.

On your PC, run:
```bash
python stream_receiver.py -d 10  # Capture 10 seconds
```

In [ ]:
import subprocess
import time

print("="*60)
print("BEFORE RUNNING: Start the receiver on your PC!")
print("  python stream_receiver.py -d <seconds>")
print("="*60)
print()

# Check for existing streaming processes
result = subprocess.run(['pgrep', '-f', 'streaming'], capture_output=True, text=True)
if result.stdout.strip():
    print("Note: Streaming process already running")
    print(f"  PIDs: {result.stdout.strip()}")
    print("  Use the cell below to stop it if needed")
else:
    print("No streaming process currently running")
    print("Ready to start streaming server")

In [ ]:
# Start the streaming server (runs in background)
# This uses the web application's streaming functionality

print("Starting streaming server...")
print(f"  Decimation: {config['decimation']} (sample rate: {sample_rate/1e6:.3f} MS/s)")
print(f"  Port: {config['port']}")
print()

# Method 1: Try using streaming-server binary
if server_path and os.path.exists(server_path):
    cmd = f"{server_path} -c {config_path} &"
    print(f"Running: {cmd}")
    os.system(cmd)
    time.sleep(2)
    print("\nStreaming server started!")
else:
    # Method 2: Use the API-based approach
    print("Using API-based streaming (may require web app to be loaded first)")
    print("")
    print("Please start streaming via the web interface:")
    print("  1. Open http://192.168.1.169/")
    print("  2. Click 'Streaming' application")
    print("  3. Set: Mode=Network, Decimation=8, Channel=CH1")
    print("  4. Click 'RUN'")

## 6. Stop Streaming Server

In [ ]:
# Stop any running streaming processes
print("Stopping streaming server...")
os.system('pkill -f streaming-server')
os.system('pkill -f rpsa_server')
time.sleep(1)

# Verify stopped
result = subprocess.run(['pgrep', '-f', 'streaming'], capture_output=True, text=True)
if result.stdout.strip():
    print(f"Warning: Process still running: {result.stdout.strip()}")
else:
    print("Streaming server stopped")

## Alternative: Use Web Interface

If the command-line server doesn't work, use the Streaming web application:

1. Open http://192.168.1.169/ in your browser
2. Click on the **Streaming** application
3. Configure:
   - **Mode:** Network
   - **Protocol:** TCP
   - **Channels:** CH1
   - **Decimation:** 8
   - **Resolution:** 16-bit
4. Click **RUN**

Then run the receiver on your PC:
```bash
python stream_receiver.py -d 60  # Capture 60 seconds
```